# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [10]:
from azureml.core import Workspace, Experiment
from azureml.core import Model 
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from pprint import pprint 
import joblib
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [11]:
#ANCLA:una vez que ya fue leida la data enviar un mensaje de realizado
#ANCLA: enviar una salida de datos del dataframe para que pueda visualizarse
#getting data: the dataset was uploaded to Azure Environment but can be accesible through an online link for example
from azureml.data.dataset_factory import TabularDatasetFactory

rawdata_homic2018 = "https://raw.githubusercontent.com/hftamayo/azuremlproj03/main/cad2018.csv"
dshomic2018 = TabularDatasetFactory.from_delimited_files(path=rawdata_homic2018, separator=',')
pdhomic2018 = dshomic2018.to_pandas_dataframe().dropna()

In [12]:
#ANCLA: hacer un check en caso que el experimento ya exista, enviar un mensaje que fue creado o que ya existe segun sea el caso
#setting up workspace and dataset
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'exp_p03_ht'

experiment=Experiment(ws, experiment_name)

In [14]:
# setting up IaaS where the experiment will run on it
ccname = "ccp03aml"
try:
    ccluster = ComputeTarget(ws, ccname)
    print(f"{ccname} exists or is in use!, choose a different name")
except:
    ccconfig = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    ccluster = ComputeTarget.create(ws, ccname, ccconfig)
ccluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [15]:
#ANCLA: enviar un mensaje que las configs fueron seteadas
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
        task='classification',
        compute_target=ccluster,
        training_data=dshomic2018,
        label_column_name='sexo',
        n_cross_validations=5,
        **automl_settings
)

In [16]:
# TODO: Submit your experiment
exprun = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [17]:
RunDetails(exprun).show()
exprun.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=====

        62   MaxAbsScaler LogisticRegression                0:00:40       0.8781    0.8814
        63   SparseNormalizer XGBoostClassifier             0:00:42       0.8799    0.8814
        64   SparseNormalizer LightGBM                      0:00:46       0.8790    0.8814
        61   SparseNormalizer XGBoostClassifier             0:01:07       0.8622    0.8814
        65   SparseNormalizer XGBoostClassifier             0:00:42       0.8790    0.8814
        67   SparseNormalizer XGBoostClassifier             0:00:46       0.8775    0.8814
        66   SparseNormalizer XGBoostClassifier             0:01:07       0.8757    0.8814
        68   SparseNormalizer XGBoostClassifier             0:00:48       0.8703    0.8814
        69   MaxAbsScaler LightGBM                          0:01:00       0.8790    0.8814
        70   SparseNormalizer XGBoostClassifier             0:00:53       0.8790    0.8814
        71                                                  0:00:14          nan    0.8814

{'runId': 'AutoML_d56f155e-e5fe-4943-949e-565227abb1c7',
 'target': 'ccp03aml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-20T12:03:37.041846Z',
 'endTimeUtc': '2021-01-20T12:35:14.722779Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'ccp03aml',
  'AMLSettingsJsonString': '{"path":null,"name":"exp_p03_ht","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-135472","workspace_name":"quick-starts-ws-135472","region":"southcentralus","compute_target":"ccp03aml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurization

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [26]:
#import azureml.train.automl
# obtaining best run and fitted model
brmodel, fmodel = exprun.get_output()

# Print the best run
print(brmodel)

# Get all metrics of the best run model
brmetrics = brmodel.get_metrics()

# Print all metrics of the best run model
for metric_name in brmetrics:
    metric = brmetrics[metric_name]
    print(metric_name, metric)
    

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


Run(Experiment: exp_p03_ht,
Id: AutoML_d56f155e-e5fe-4943-949e-565227abb1c7_74,
Type: azureml.scriptrun,
Status: Completed)
matthews_correlation 0.15632406477695038
precision_score_weighted 0.8806403332276602
recall_score_macro 0.5181587663709311
recall_score_micro 0.8822475068603172
average_precision_score_macro 0.5666532453806017
weighted_accuracy 0.980698779293733
precision_score_macro 0.8735382623166542
norm_macro_recall 0.0363175327418622
balanced_accuracy 0.5181587663709311
AUC_macro 0.6156157887216929
AUC_micro 0.9034464488814242
recall_score_weighted 0.8822475068603172
f1_score_macro 0.5041450478296661
average_precision_score_weighted 0.829086142755828
log_loss 0.3610046604665561
precision_score_micro 0.8822475068603172
accuracy 0.8822475068603172
AUC_weighted 0.6156157887216929
average_precision_score_micro 0.8867503151868703
f1_score_weighted 0.8321888922619733
f1_score_micro 0.8822475068603172
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_d56f155e-e5fe-4943-949e-

In [27]:
#TODO: Save the best model
autobmlp3ht = brmodel.register_model(model_path='outputs/model.pkl', model_name='p3automl_ht',
                        tags={'Method of execution':'Auto ML'},
                        properties={'Accuracy': brmetrics['accuracy']})

print(autobmlp3ht)

Model(workspace=Workspace.create(name='quick-starts-ws-135472', subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f', resource_group='aml-quickstarts-135472'), name=p3automl_ht, id=p3automl_ht:1, version=1, tags={'Method of execution': 'Auto ML'}, properties={'Accuracy': '0.8822475068603172'})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [28]:
# Download scoring file 
brmodel.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')

# Download environment file
brmodel.download_file('outputs/conda_env_v_1_0_0.yml', 'enviro.yml')

In [29]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

infcfg = InferenceConfig(entry_script='scoring.py',
                                    environment=Environment.from_conda_specification(name='myenv',file_path='enviro.yml'))

# deploying the model via WebService
from azureml.core.webservice import AciWebservice

depcfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
webservice = Model.deploy(ws, "wsp03ht01", [autobmlp3ht], infcfg, depcfg)
webservice.wait_for_deployment(show_output = True)
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...................................................................................................................................................................................................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: f8e43ff6-5a8c-46d4-bc47-392bf11226e9
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: wsp03ht01. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image 7259fd9b8cea46f4b0e830dd1ea4ead2.azurecr.io/azureml/azureml_99f2dd0e6dff5bc19133b480a94a6686 locally. Please refer to https://aka.ms/debugimage#service-lau

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: f8e43ff6-5a8c-46d4-bc47-392bf11226e9
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: wsp03ht01. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image 7259fd9b8cea46f4b0e830dd1ea4ead2.azurecr.io/azureml/azureml_99f2dd0e6dff5bc19133b480a94a6686 locally. Please refer to https://aka.ms/debugimage#

In [ ]:
import json

#Importing the data set for testing 
tsthomic2018 = pdhomic2018.sample(5) 
lblhomic2018 = tsthomic2018.pop('sexo')

tstdatahomic2018 = json.dumps({'pdhomic2018': tsthomic2018.to_dict(orient='records')})

print(tstdatahomic2018)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests

headers = {'Content-type': 'application/json'}

response = requests.post(webservice.scoring_uri, tstdatahomic2018, headers=headers)

In [ ]:
# Print results from the inference
print(response.text)

In [ ]:
# Print original labels
print(lblhomic2018)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(webservice.get_logs())

In [ ]:
webservice.delete()